In [37]:
from sqlalchemy import event, create_engine, Column, Integer, ForeignKey
from sqlalchemy.orm import declarative_base, sessionmaker, relationship, selectinload, with_loader_criteria
from sqlalchemy.sql.expression import select


Base = declarative_base()
class Foo(Base):
    __tablename__ = "foo"
    id = Column(Integer, primary_key=True)
    foo = Column(Integer)
    bars = relationship('Bar')

class Bar(Base):
    __tablename__ = "bar"
    id = Column(Integer, primary_key=True)
    bar = Column(Integer)
    foo_id = Column(Integer, ForeignKey('foo.id'))
    foo = relationship(Foo, back_populates='bars')


engine = create_engine("sqlite://", future=True)
Session = sessionmaker()
Base.metadata.create_all(bind=engine)

connection = engine.connect()
txn = connection.begin()
session = Session(bind=connection)

session.add(foo1 := Foo(foo=1))
session.add(foo2 := Foo(foo=2))
session.add(Bar(bar=1, foo=foo1))
session.add(Bar(bar=2, foo=foo2))
session.commit()

q = select(Foo).options(selectinload(Foo.bars), with_loader_criteria(Bar, Bar.bar == 1))
result = session.execute(q).scalars().all()
print(result[0].bars, result[1].bars)  # [] []

[<__main__.Bar object at 0x7f80856b7100>] []
